# Analysons des factures d'énergie

In [56]:
import pandas as pd
import numpy as np 
import re

data = pd.read_csv('Invoice_20201220.csv', sep='|')
data

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,12345_GEO (0001BB),1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,12345_GEO (0001BB),1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,12345_GEO (0001BB),1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,12345_GEO (0001BB),1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,12345_GEO (0001BB),1300082549,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...,...
51427,19767_GEO (0632KO),FV1/055100030,30/08/2019,31/12/2019,3 028,"397,18"
51428,19767_GEO (0632KO),055100058,01/01/2020,31/03/2020,2 163,"283,57"
51429,19767_GEO (0632KO),055100113,01/04/2020,30/06/2020,2 160,"312,55"
51430,19767_GEO (0632KO),055100168,01/07/2020,30/09/2020,2 230,"294,07"


### 1) Le nom d'une antenne a un format bien précis : il est toujours composé de 4 chiffres et de deux lettres. On te demande de retirer tout autre caractère de la colonne "SITE_NAME".

In [57]:
data['SITE_NAME'] = data['SITE_NAME'].str[11:].str[:-1]
data

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,1300082549,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...,...
51427,0632KO,FV1/055100030,30/08/2019,31/12/2019,3 028,"397,18"
51428,0632KO,055100058,01/01/2020,31/03/2020,2 163,"283,57"
51429,0632KO,055100113,01/04/2020,30/06/2020,2 160,"312,55"
51430,0632KO,055100168,01/07/2020,30/09/2020,2 230,"294,07"


### 2) On te demande de supprimer la colonne "INVOICE_NAME" qui n'apporte rien aux équipes.

In [58]:
data.drop('INVOICE_NAME', axis=1, inplace=True)
data

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...
51427,0632KO,30/08/2019,31/12/2019,3 028,"397,18"
51428,0632KO,01/01/2020,31/03/2020,2 163,"283,57"
51429,0632KO,01/04/2020,30/06/2020,2 160,"312,55"
51430,0632KO,01/07/2020,30/09/2020,2 230,"294,07"


### 3) Il y a des antennes pour lesquelles on a des factures mensuelles tandis que pour d'autres, on les reçoit moins régulièrement (trimestre, semestre, année). Ajoute une colonne à ton tableau qui calcule le nombre de jours entre le début de la période de facturation et la fin de la période de facturation.

In [59]:
data.BEGIN = pd.to_datetime(data.BEGIN)
data.END = pd.to_datetime(data.END)
data['PERIODE_FACTURATION'] = (data.END - data.BEGIN)
data

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,PERIODE_FACTURATION
0,0001BB,2018-01-01,2018-01-31,2 447,"316,46",30 days
1,0001BB,2018-01-02,2018-02-28,2 211,"286,82",57 days
2,0001BB,2018-01-03,2018-03-31,2 450,"317,32",87 days
3,0001BB,2018-01-04,2018-04-30,2 431,"315,00",116 days
4,0001BB,2018-01-05,2018-05-31,2 574,"333,37",146 days
...,...,...,...,...,...,...
51427,0632KO,2019-08-30,2019-12-31,3 028,"397,18",123 days
51428,0632KO,2020-01-01,2020-03-31,2 163,"283,57",90 days
51429,0632KO,2020-01-04,2020-06-30,2 160,"312,55",178 days
51430,0632KO,2020-01-07,2020-09-30,2 230,"294,07",267 days


### 4) A partir de cette nouvelle colonne, crée une autre colonne qui nous donne la consommation journalière moyenne de la période.

In [60]:
data['CONSO_JOURNALIERE'] = ((data['KWH_PERIOD'].replace(' ','', regex=True).astype(str).astype(int)) / data['PERIODE_FACTURATION'].dt.days).round(decimals = 2)
data

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,PERIODE_FACTURATION,CONSO_JOURNALIERE
0,0001BB,2018-01-01,2018-01-31,2 447,"316,46",30 days,81.57
1,0001BB,2018-01-02,2018-02-28,2 211,"286,82",57 days,38.79
2,0001BB,2018-01-03,2018-03-31,2 450,"317,32",87 days,28.16
3,0001BB,2018-01-04,2018-04-30,2 431,"315,00",116 days,20.96
4,0001BB,2018-01-05,2018-05-31,2 574,"333,37",146 days,17.63
...,...,...,...,...,...,...,...
51427,0632KO,2019-08-30,2019-12-31,3 028,"397,18",123 days,24.62
51428,0632KO,2020-01-01,2020-03-31,2 163,"283,57",90 days,24.03
51429,0632KO,2020-01-04,2020-06-30,2 160,"312,55",178 days,12.13
51430,0632KO,2020-01-07,2020-09-30,2 230,"294,07",267 days,8.35


### 5) Merci d'ajouter également une colonne qui donne, pour chaque antenne, le nombre de factures dont on dispose au total. Pas grave si l'information se répète à chaque ligne pour une même antenne.

In [61]:
data['NB_INVOICES_SITE'] = data['SITE_NAME'].map(data['SITE_NAME'].value_counts())
data

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,PERIODE_FACTURATION,CONSO_JOURNALIERE,NB_INVOICES_SITE
0,0001BB,2018-01-01,2018-01-31,2 447,"316,46",30 days,81.57,35
1,0001BB,2018-01-02,2018-02-28,2 211,"286,82",57 days,38.79,35
2,0001BB,2018-01-03,2018-03-31,2 450,"317,32",87 days,28.16,35
3,0001BB,2018-01-04,2018-04-30,2 431,"315,00",116 days,20.96,35
4,0001BB,2018-01-05,2018-05-31,2 574,"333,37",146 days,17.63,35
...,...,...,...,...,...,...,...,...
51427,0632KO,2019-08-30,2019-12-31,3 028,"397,18",123 days,24.62,4
51428,0632KO,2020-01-01,2020-03-31,2 163,"283,57",90 days,24.03,4
51429,0632KO,2020-01-04,2020-06-30,2 160,"312,55",178 days,12.13,4
51430,0632KO,2020-01-07,2020-09-30,2 230,"294,07",267 days,8.35,4


### 6) Et information bonus si tu y arrives : les équipes aimeraient beaucoup avoir une liste de toutes les antennes avec, pour chaque antenne, le coefficient de variation de la consommation. Cela permettra d'avoir une idée, pour chaque antenne, de la dispersion de sa consommation d'énergie.

In [62]:
data.pivot_table('CONSO_JOURNALIERE', index='SITE_NAME', aggfunc=[np.std,np.mean]).to_csv('tcd.csv')

In [63]:
# remplacer les points par des virgules puis diviser std par mean dans une nouvelle colonne pour obtenir le coefficient de variation.

### 7) Enfin, tu dois exporter ces deux fichiers obtenus pour pouvoir l'envoyer aux équipes de l'opérateur et leur demander si c'est bien cela qu'ils attendaient de toi.

In [64]:
data.to_csv('exo1.csv')

In [65]:
data.pivot_table('CONSO_JOURNALIERE', index='SITE_NAME', aggfunc=[np.std,np.mean])

,std,mean
,CONSO_JOURNALIERE,CONSO_JOURNALIERE
SITE_NAME,,
,15.038896,14.804545
0001BB,23.847285,24.847143
0001BR,21.139377,22.438857
0001KO,20.574752,22.521143
0002BB,23.378151,24.358857
...,...,...
1442BR,NaN,33.160000
1481BR,NaN,41.010000
